# 中等项目2：多特征回归优化

## 项目描述

重点练习特征工程和模型优化。这是一个中等难度的项目，帮助我们理解如何通过特征选择和超参数调优来提升模型性能。

## 学习目标

通过本项目，你将学会：
1. 如何处理类别特征
2. 如何进行特征选择
3. 如何使用GridSearchCV进行超参数调优
4. 如何对比优化前后的模型性能
5. 如何分析特征重要性

## 项目流程

1. 数据生成：生成多特征数据（包含类别特征）
2. 数据预处理：处理类别特征、标准化
3. 特征选择：使用SelectKBest选择重要特征
4. 模型训练和优化：对比优化前后的模型性能
5. 结果可视化：可视化优化效果


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置matplotlib在notebook中内联显示
%matplotlib inline

print("环境准备完成！")


## 第一步：生成数据

生成多特征数据，包括数值特征和类别特征。注意：只有部分特征真正有用。


In [ ]:
def generate_data(n_samples=300, random_state=42):
    """
    生成多特征数据
    
    参数:
    - n_samples: 样本数量
    - random_state: 随机种子
    
    返回:
    - df: 包含特征和目标的DataFrame
    """
    np.random.seed(random_state)
    
    data = {
        'feature1': np.random.normal(10, 2, n_samples),
        'feature2': np.random.normal(20, 3, n_samples),
        'feature3': np.random.normal(5, 1, n_samples),
        'feature4': np.random.randint(0, 10, n_samples),
        'feature5': np.random.choice(['A', 'B', 'C'], n_samples),  # 类别特征
        'feature6': np.random.uniform(0, 1, n_samples),
        'feature7': np.random.poisson(3, n_samples),
        'feature8': np.random.normal(15, 2, n_samples),
        'feature9': np.random.choice([0, 1], n_samples),
        'feature10': np.random.normal(8, 1.5, n_samples)
    }
    
    df = pd.DataFrame(data)
    
    # 目标：只有部分特征真正有用
    df['target'] = (
        df['feature1'] * 2 +
        df['feature2'] * 1.5 +
        df['feature3'] * 3 +
        df['feature5'].map({'A': 5, 'B': 3, 'C': 1}) +
        df['feature9'] * 4 +
        np.random.normal(0, 5, n_samples)
    )
    
    return df

# 生成数据
df = generate_data(n_samples=300)

print("=" * 70)
print("数据生成完成")
print("=" * 70)
print(f"数据形状: {df.shape}")
print(f"\n数据前5行:")
print(df.head())
print(f"\n特征说明:")
print(f"  真正有用的特征: feature1, feature2, feature3, feature5, feature9")
print(f"  噪声特征: feature4, feature6, feature7, feature8, feature10")


## 第二步：数据预处理

处理类别特征（使用LabelEncoder），分离特征和目标，划分训练集和测试集，标准化特征。


In [ ]:
# 分离特征和目标
X = df.drop('target', axis=1)
y = df['target']

# 处理类别特征
le = LabelEncoder()
X_processed = X.copy()
X_processed['feature5_encoded'] = le.fit_transform(X['feature5'])
X_processed = X_processed.drop('feature5', axis=1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=" * 70)
print("数据预处理完成")
print("=" * 70)
print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征数量: {X_train.shape[1]}")
print(f"\n类别特征编码:")
print(f"  feature5: {dict(zip(le.classes_, le.transform(le.classes_)))}")


## 第三步：特征选择

使用SelectKBest选择最重要的5个特征。


In [ ]:
print("=" * 70)
print("特征选择")
print("=" * 70)

# 特征选择
selector = SelectKBest(score_func=f_regression, k=5)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

selected_features = X_train.columns[selector.get_support()]
print(f"\n选择的特征: {list(selected_features)}")
print(f"选择的特征数量: {len(selected_features)}")

# 显示特征得分
feature_scores = pd.DataFrame({
    '特征': X_train.columns,
    '得分': selector.scores_,
    '是否选择': selector.get_support()
}).sort_values('得分', ascending=False)

print(f"\n特征得分排序:")
print(feature_scores.to_string(index=False))


## 第四步：模型训练和优化

对比优化前（使用所有特征）和优化后（特征选择+超参数调优）的模型性能。


In [ ]:
print("=" * 70)
print("模型训练和优化")
print("=" * 70)

# 4.1 优化前（使用所有特征）
models_before = {
    '线性回归': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1)
}

print("\n优化前（使用所有特征）:")
print(f"{'模型':<20} {'测试MSE':<15} {'测试R²':<15}")
print("-" * 50)

results_before = {}
for name, model in models_before.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results_before[name] = {'mse': mse, 'r2': r2}
    print(f"{name:<20} {mse:<15.4f} {r2:<15.4f}")


### 4.2 优化后（特征选择+超参数调优）

使用GridSearchCV进行超参数调优。


In [ ]:
# 4.2 优化后（特征选择+超参数调优）
print("\n优化后（特征选择+超参数调优）:")

# Ridge调优
ridge_param_grid = {'alpha': np.logspace(-2, 2, 20)}
ridge_grid = GridSearchCV(Ridge(), ridge_param_grid, cv=5, scoring='r2', n_jobs=-1)
ridge_grid.fit(X_train_selected, y_train)

# Lasso调优
lasso_param_grid = {'alpha': np.logspace(-3, 1, 20)}
lasso_grid = GridSearchCV(Lasso(max_iter=10000), lasso_param_grid, cv=5, scoring='r2', n_jobs=-1)
lasso_grid.fit(X_train_selected, y_train)

models_after = {
    '线性回归': LinearRegression(),
    'Ridge (优化)': ridge_grid.best_estimator_,
    'Lasso (优化)': lasso_grid.best_estimator_
}

print(f"{'模型':<20} {'测试MSE':<15} {'测试R²':<15} {'改进':<15}")
print("-" * 65)

for name, model in models_after.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 计算改进
    if name.replace(' (优化)', '') in results_before:
        base_name = name.replace(' (优化)', '')
        improvement = r2 - results_before[base_name]['r2']
        print(f"{name:<20} {mse:<15.4f} {r2:<15.4f} {improvement:+.4f}")
    else:
        print(f"{name:<20} {mse:<15.4f} {r2:<15.4f} {'N/A':<15}")

print(f"\n最佳Ridge参数: alpha={ridge_grid.best_params_['alpha']:.4f}")
print(f"最佳Lasso参数: alpha={lasso_grid.best_params_['alpha']:.4f}")


## 第五步：结果可视化

可视化优化前后的性能对比和特征重要性。


In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 性能对比
model_names = list(results_before.keys())
r2_before = [results_before[m]['r2'] for m in model_names]
r2_after = [r2_score(y_test, models_after[m].predict(X_test_selected)) 
            for m in model_names]

x = np.arange(len(model_names))
width = 0.35
axes[0].bar(x - width/2, r2_before, width, label='优化前', alpha=0.7, color='lightcoral')
axes[0].bar(x + width/2, r2_after, width, label='优化后', alpha=0.7, color='skyblue')
axes[0].set_xlabel('模型', fontsize=12)
axes[0].set_ylabel('R²分数', fontsize=12)
axes[0].set_title('优化前后性能对比', fontsize=14)
axes[0].set_xticks(x)
axes[0].set_xticklabels(model_names)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# 特征重要性（Lasso）
best_lasso = lasso_grid.best_estimator_
selected_feature_names = X_train.columns[selector.get_support()]
coef = best_lasso.coef_

axes[1].barh(range(len(coef)), coef, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1].set_yticks(range(len(coef)))
axes[1].set_yticklabels(selected_feature_names)
axes[1].set_xlabel('系数值', fontsize=12)
axes[1].set_title('Lasso特征重要性（优化后）', fontsize=14)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n观察：")
print("- 优化后的模型性能通常更好或相当")
print("- 特征选择减少了特征数量，提高了模型可解释性")
print("- 超参数调优找到了更好的正则化参数")


## 第六步：项目总结

总结优化效果和关键发现。


In [ ]:
print("=" * 70)
print("项目总结")
print("=" * 70)

print("\n优化效果总结:")
print(f"{'模型':<20} {'优化前R²':<15} {'优化后R²':<15} {'改进':<15}")
print("-" * 65)
for name in model_names:
    before_r2 = results_before[name]['r2']
    after_r2 = r2_score(y_test, models_after[name].predict(X_test_selected))
    improvement = after_r2 - before_r2
    print(f"{name:<20} {before_r2:<15.4f} {after_r2:<15.4f} {improvement:+.4f}")

print(f"\n选择的特征: {list(selected_features)}")
print(f"特征数量: {len(selected_features)} (从 {X_train.shape[1]} 个特征中选择)")

print("\n关键发现:")
print("1. 特征选择可以去除噪声特征，提高模型性能")
print("2. 超参数调优可以找到更好的正则化参数")
print("3. 优化后的模型通常更简洁、更易解释")
print("4. Lasso回归在特征选择方面表现突出")

print("\n✅ 项目完成！")
